In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
offshore = xr.open_dataset('/lsdf/kit/imk-tro/projects/Gruppe_Quinting/om1434/offshore/offshore.zarr', engine='zarr')
offshore

Derive a land sea mask out of the energy data by picking all null values. Then make sure they look all the same by picking 50 random timepoints and copare them

In [ ]:
dataarray = offshore.sel(time='2009-01-01T00:00:00').wofcfr
dataarray = dataarray.drop_vars('time')

lsm_derived = offshore.sel(time='2009-01-01T00:00:00').wofcfr.isnull().astype(int)
lsm_derived = lsm_derived.rename('lsm')
lsm_derived = lsm_derived.drop_vars('time')

In [ ]:
# Pick 5 random timestamps
timestamps = np.random.choice(offshore.time.values, 2, replace=False)

for i, ts in enumerate(timestamps):
    lsm_derived_ts = offshore.sel(time=ts).wofcfr.isnull().astype(int)
    # Compare numerically
    assert np.array_equal(lsm_derived, lsm_derived_ts), f"Mismatch at timestamp {ts}"

In [ ]:
lsm_derived.to_zarr('/lsdf/kit/imk-tro/projects/Gruppe_Quinting/om1434/offshore/lsm_derived.zarr', mode='w')

Expand coordinate system to fit era5

In [ ]:
# Expand the latitude and longitude dimensions to cover the whole globe
expanded_latitude = np.arange(-90, 90.25, 0.25)
expanded_longitude = np.arange(-180, 180, 0.25)

# Reindex the lsm_derived to the new global coordinates. Method=None: fill the gaps with NAs
lsm_global = lsm_derived.reindex(latitude=expanded_latitude, longitude=expanded_longitude, method=None)
# Roll the longitudes [-180, 180) -> [0, 360) - by 180 degrees
lsm_global = lsm_global.roll(longitude=180*4, roll_coords=True)
# Convert negative longitudes to positive
lsm_global.coords['longitude'] = (lsm_global.coords['longitude'] + 360) % 360

lsm_global.plot()

In [ ]:
lsm_global.to_zarr('/lsdf/kit/imk-tro/projects/Gruppe_Quinting/om1434/offshore/lsm_global.zarr', mode='w')

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

lsm = xr.load_dataarray("/lsdf/kit/imk-tro/projects/Gruppe_Quinting/om1434/offshore/lsm_global.zarr", engine="zarr")
lsm = xr.where(lsm.isnull(), float("nan"), xr.where(lsm == 0, 1, float("nan"))).values
# lsm = np.flipud(lsm).copy()

lsm_xr = xr.DataArray(lsm, dims=["latitude", "longitude"])
fig, ax = plt.subplots(figsize=(12/2, 8/2), dpi=150)
lsm_xr.plot(ax=ax)
plt.show()

Cut out left part of europe

In [ ]:
lsm_cut = lsm[466 : 651, :183]
print(lsm_cut.shape)

lsm_xr = xr.DataArray(lsm_cut, dims=["latitude", "longitude"])
fig, ax = plt.subplots(figsize=(12/2, 8/2), dpi=150)
lsm_xr.plot(ax=ax)
print(lsm_cut[184,182])